In [1]:
import pandas as pd
import requests
import os

In [2]:
# --- 1. CONFIGURATION ---
OUTPUT_DIR = "output"
OUTPUT_FILE = "nyc_census_tract_demographics.csv"

# ACS variable codes for Total Population and Median Household Income
ACS_VARS = {
    "B01003_001E": "total_population",
    "B19013_001E": "median_income"
}

In [3]:
# --- 2. CONSTRUCT AND EXECUTE CENSUS API CALL ---
print("➡️ Step 1: Requesting data from the US Census API...")

# Construct the API URL to get the variables for all tracts in New York State (FIPS code 36)
var_string = ",".join(ACS_VARS.keys())
api_url = f"https://api.census.gov/data/2022/acs/acs5?get={var_string},NAME&for=tract:*&in=state:36"

# Make the request and convert the JSON response to a DataFrame
response = requests.get(api_url)
data = response.json()
df_acs = pd.DataFrame(data[1:], columns=data[0])

print(f"✅ Data for {len(df_acs)} tracts in NY State loaded successfully.")

➡️ Step 1: Requesting data from the US Census API...
✅ Data for 5411 tracts in NY State loaded successfully.


In [4]:
# --- 3. FILTER TO NYC BOROUGHS ---
print("\n➡️ Step 2: Filtering data to the five NYC boroughs...")

# FIPS codes for the 5 NYC counties
nyc_county_fips = ['005', '047', '061', '081', '085'] # Bronx, Kings, New York, Queens, Richmond
df_acs_nyc = df_acs[df_acs['county'].isin(nyc_county_fips)].copy()

print(f"✅ Data filtered to NYC-only tracts: {len(df_acs_nyc)} remaining.")


➡️ Step 2: Filtering data to the five NYC boroughs...
✅ Data filtered to NYC-only tracts: 2327 remaining.


In [5]:
# --- 4. CLEAN DATA AND FORMAT TRACT IDS ---
print("\n➡️ Step 3: Cleaning data and creating custom tract IDs...")

# Rename the columns to be more descriptive
df_acs_nyc = df_acs_nyc.rename(columns=ACS_VARS)

# Define the mapping from county FIPS to our custom borough digit
county_to_borough_digit = {
    '061': '1',  # Manhattan (New York County)
    '005': '2',  # Bronx
    '047': '3',  # Brooklyn (Kings County)
    '081': '4',  # Queens
    '085': '5'   # Staten Island (Richmond County)
}

# Create the custom 7-digit tract ID to match your other files
df_acs_nyc['tract_id'] = df_acs_nyc['county'].map(county_to_borough_digit) + df_acs_nyc['tract']
print("✅ Custom tract IDs created.")

# Convert population and income to numeric types.
# The API returns negative values for missing/suppressed data, so we'll turn those into NaN (Not a Number).
for col in ['total_population', 'median_income']:
    df_acs_nyc[col] = pd.to_numeric(df_acs_nyc[col], errors='coerce')
    df_acs_nyc.loc[df_acs_nyc[col] < 0, col] = None # Set negative values to NaN

# Keep only the essential, final columns
df_final = df_acs_nyc[['tract_id', 'total_population', 'median_income']]
print("✅ Data cleaned and formatted.")


➡️ Step 3: Cleaning data and creating custom tract IDs...
✅ Custom tract IDs created.
✅ Data cleaned and formatted.


In [6]:
# --- 5. SAVE THE FINAL CSV ---
print("\n➡️ Step 4: Saving the final demographic data CSV...")
os.makedirs(OUTPUT_DIR, exist_ok=True)
output_path = os.path.join(OUTPUT_DIR, OUTPUT_FILE)
df_final.to_csv(output_path, index=False)

print(f"✅ Final demographics file saved to: '{output_path}'")
print("\n--- Sample of the Final Demographics Data ---")
print(df_final.head())


➡️ Step 4: Saving the final demographic data CSV...
✅ Final demographics file saved to: 'output/nyc_census_tract_demographics.csv'

--- Sample of the Final Demographics Data ---
    tract_id  total_population  median_income
101  2000100            4446.0            NaN
102  2000200            4870.0       115064.0
103  2000400            6257.0       100553.0
104  2001600            6177.0        41362.0
105  2001901            2181.0        49500.0
